<a href="https://colab.research.google.com/github/dominikacecylia/Automatic-post-editing/blob/main/Multi_Open_APE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPU Checks

In [ ]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
    process = psutil.Process(os.getpid())
    print("CPU RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Created wheel for gputil: filename=GPUtil-1.4.0-cp37-none-any.whl size=7411 sha256=10f29f8229c06f0c841841f4bc0a2696afdb719ba71c2addd545bf9d4f36009a
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
CPU RAM Free: 26.4 GB  | Proc size: 111.8 MB
GPU RAM Free: 16160MB | Used: 0MB | Util   0% | Total 16160MB


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Wed Mar  3 10:27:22 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Requirements

### Downloading Libraries 
which are required to be present on the system for OpenNMT-APE to work:
- Pretrained BERT
- configargparse
- cu100 torch 1.0.01
- torchvision 0.2.2
- tqdm 4.19.9

In [ ]:
! pip uninstall fbprophet -y
! pip uninstall fastai -y
! pip uninstall spacy -y

Uninstalling fbprophet-0.7.1:
  Successfully uninstalled fbprophet-0.7.1
Uninstalling fastai-1.0.61:
  Successfully uninstalled fastai-1.0.61
Uninstalling spacy-2.2.4:
  Successfully uninstalled spacy-2.2.4


In [ ]:
! pip install tqdm==4.19.9
! pip install torchvision==0.2.2
! pip install torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html
! pip install pytorch_pretrained_bert configargparse
! pip install torchtext==0.6.0

     |████████████████████████████████| 61kB 6.5MB/s 
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1
     |████████████████████████████████| 71kB 6.2MB/s 
  Found existing installation: torchvision 0.8.2+cu101
    Uninstalling torchvision-0.8.2+cu101:
      Successfully uninstalled torchvision-0.8.2+cu101
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 704.4MB 24kB/s 
     |████████████████████████████████| 6.7MB 43.0MB/s 
  Found existing installation: torch 1.7.1+cu101
    Uninstalling torch-1.7.1+cu101:
      Successfully uninstalled torch-1.7.1+cu101
  Found existing installation: torchvision 0.2.2
    Uninstalling torchvision-0.2.2:
      Successfully uninstalled torchvision-0.2.2
     |████████████████████████████████| 133kB 14.3MB/s 
     |████████████████████████████████| 51kB 7.6MB/s 
     |████████████████████████████████| 133kB 22.5MB/s 
     |███

### Downloading OpenNMT-APE

In [ ]:
! git clone https://github.com/deep-spin/OpenNMT-APE

# Imports

In [ ]:
import torch
import random
from copy import copy
import numpy as np
torch.cuda.empty_cache()
import os
import json
import sys
import random
from pprint import pprint
from google.colab import files
import logging
logging.root.setLevel(logging.DEBUG)
! rm -rf sample_data

# Uploader

In [ ]:
def uploader():
    required = sorted(['mt.txt', 'pe.txt'])
    logging.info(f'please upload the following: {required}')
    uploaded = files.upload()
    uploaded = sorted([*uploaded.keys()])
    if uploaded != required:
        logging.warning(f'uploaded: {uploaded}, required: {required}')
    else:
        logging.info(f'uploaded: {uploaded}')

# Resetter

In [ ]:
def resetter():
    paths = [
             'OpenNMT-APE',
             'OpenNMT-py',
             'preprocessing.yml',
             'train-config.yml',
             'mosesdecoder'
    ]
    msg = "\n"+"\n".join(paths)+"\n"
    choice = input(f'would you like to reset [y/n]: {msg}?\n')
    if choice == 'y':
        for path in paths:
            ! rm -rf $path

# Train/Test Splitter

In [ ]:
def train_test_split_without_keys(mtname: 'mt file name', 
                                  pename: 'pe file name', 
                                  t_ratio: 'training ratio, between [0, 1]'):
    num = 0
    mtlines = []
    pelines = []
    with open(mtname, 'r') as mtfile:
        mtlines = mtfile.readlines()
        num = len(mtlines)
        mtlines = [line.strip() for line in mtlines]
        mtlines = np.array(mtlines)
    with open(pename, 'r') as pefile:
        pelines = pefile.readlines()
        pelines = [line.strip() for line in pelines]
        pelines = np.array(pelines)
    t_num = int(t_ratio*num)
    keys = [*range(num)]
    train_keys = random.sample(copy(keys), t_num)
    test_keys  = [key for key in keys if key not in train_keys]
    trainmt    = mtlines[train_keys]
    testmt     = mtlines[test_keys]
    trainpe    = pelines[train_keys]
    testpe     = pelines[test_keys]
    trainmt  = "\n".join(trainmt)
    testmt   = "\n".join(testmt)
    trainpe  = "\n".join(trainpe)
    testpe   = "\n".join(testpe)
    with open('mt_train.txt', 'w') as fileobj:
        fileobj.write(trainmt)
    with open('mt_test.txt', 'w') as fileobj:
        fileobj.write(testmt)
    with open('pe_train.txt', 'w') as fileobj:
        fileobj.write(trainpe)
    with open('pe_test.txt', 'w') as fileobj:
        fileobj.write(testpe)
    trainkeys = ["" for _ in range(len(train_keys))]
    testkeys  = ["" for _ in range(len(test_keys))]
    with open('trainkeys.txt', 'w') as fileobj:
        json.dump(trainkeys, fileobj)
    with open('testkeys.txt', 'w') as fileobj:
        json.dump(testkeys, fileobj)


def train_test_split_with_keys(mtname: 'mt file name', 
                                pename: 'pe file name', 
                                t_ratio: 'training ratio, between [0, 1]'):
    mtdict = {}
    pedict = {}
    with open(mtname, 'r') as mtfile:
        mtlines = mtfile.readlines()
        for line in mtlines:
            linearray = line.split()
            key       = linearray[0]
            value     = " ".join(linearray[1:])
            mtdict[key] = value
    with open(pename, 'r') as pefile:
        pelines = pefile.readlines()
        for line in pelines:
            linearray = line.split()
            key       = linearray[0]
            value     = " ".join(linearray[1:])
            pedict[key] = value
    assert len([*mtdict.keys()]) == len([*pedict.keys()])
    keys = [*mtdict.keys()]
    random.shuffle(keys)
    num = int(t_ratio*len(keys))
    trainkeys = keys[:num]
    testkeys  = keys[num:]
    trainmt   = [
                mtdict[key]
                for key in trainkeys
    ]
    testmt   = [
                mtdict[key]
                for key in testkeys
    ]
    trainpe   = [
                pedict[key]
                for key in trainkeys
    ]
    testpe   = [
                pedict[key]
                for key in testkeys
    ]
    trainmt  = "\n".join(trainmt)
    testmt   = "\n".join(testmt)
    trainpe  = "\n".join(trainpe)
    testpe   = "\n".join(testpe)
    with open('mt_train.txt', 'w') as fileobj:
        fileobj.write(trainmt)
    with open('mt_test.txt', 'w') as fileobj:
        fileobj.write(testmt)
    with open('pe_train.txt', 'w') as fileobj:
        fileobj.write(trainpe)
    with open('pe_test.txt', 'w') as fileobj:
        fileobj.write(testpe)
    with open('trainkeys.txt', 'w') as fileobj:
        json.dump(trainkeys, fileobj)
    with open('testkeys.txt', 'w') as fileobj:
        json.dump(testkeys, fileobj)


def train_test_split(mtname: 'mt file name', 
                     pename: 'pe file name', 
                     t_ratio: 'training ratio, between [0, 1]',
                     idflag: 'whether the files contain ids or not'):
    if idflag:
        train_test_split_with_keys(mtname, pename, t_ratio)
    else:
        train_test_split_without_keys(mtname, pename, t_ratio)


### Cleanup Script

In [ ]:
def cleanup(filename: 'file name of the ape file', mtname):
    result = ""
    trainkeys, testkeys = None, None
    with open('trainkeys.txt', 'r') as fileobj:
        trainkeys = json.load(fileobj)
        assert type(trainkeys) == list
    with open('testkeys.txt', 'r') as fileobj:
        testkeys = json.load(fileobj)
        assert type(testkeys) == list
    missinglines = []
    missingkeys  = []
    mtlines = []
    with open(mtname, 'r') as mtfile:
        mtlines = mtfile.readlines()
    with open(filename, 'r') as fileobject:
        lines = fileobject.readlines()
        for line, key in zip(lines, testkeys):
            result += key + " "
            linearray = line.split()
            if len(linearray) <= 1:
                missinglines.append(mtlines[number].stript())
                missingkeys.append(line.strip())
            for i in range(len(linearray)):
                if linearray[i].startswith("##"):
                    result += linearray[i][2:]
                elif i > 0:
                    result += " " + linearray[i]
                else:
                    result += linearray[i]
            result += "\n"
    with open(filename, 'w') as fileobject:
        fileobject.write(result)
    result = {
        "missing lines": missinglines,
        "missing keys": missingkeys
    }
    print(result)
    missinglines   = "\n".join(missinglines)
    with open('missing_mt.txt', 'w') as fileobj:
        fileobj.write(missinglines)

# Upload Dataset
Upload the following two files together:
- mt.txt
- pe.txt

In [ ]:
!ls

drive


In [ ]:
uploader()

INFO:root:please upload the following: ['mt.txt', 'pe.txt']


Saving train_mt_new.txt to train_mt_new.txt
Saving train_pe_new.txt to train_pe_new.txt


In [ ]:
files1 = ['train_mt_new.txt', 'train_pe_new.txt']
for file1 in files1:
  f = open(file1, 'r') 
  lines = f.readlines()
  newlines = []
  length = 60
  for line in lines:
    tokens = line.split()
    if len(tokens) > length:
      for i in range(len(tokens)//length + 1):
       buf = [tokens[0]]
       buf.extend(tokens[i*length+1:(i+1)*length+1])
       l = ' '.join(buf) + '\n'
       if l != '\n':
          newlines.append(l)
    else:
      newlines.append(' '.join(tokens) + '\n')
  outF = open(file1, 'w')
  outF.writelines(newlines)
  outF.close()

In [ ]:
!ls

drive  train_mt.txt  train_pe.txt


# User Input

**Optimal Parameter Values, as per the OpenNMT-APE Documentation:**
- train_steps: 50000
- start_decay_steps: 50000
- valid_steps: 1000
- keep_checkpoint: 30
- warmup_steps: 5000
- learning_rate: 0.00005
- average_decay: 0.0001
- src_seq_length: 100
- tgt_seq_length: 100

**Other Parameters**:
- **idflag**: check the box if the datasets contain ids
- **t_ratio**: training ratio, between [0, 1], preferred to be larger than 0.5.

In [ ]:
#@title Parameter Input
train_steps = 50000 #@param {type:"integer"}
start_decay_steps = 50000 #@param {type:"integer"}
valid_steps = 5000 #@param {type:"integer"}
keep_checkpoint = 30 #@param {type:"integer"}
warmup_steps = 5000 #@param {type:"integer"}
learning_rate = 0.00005 #@param {type:"number"}
average_decay = 0.0001 #@param {type:"number"}
src_seq_length = 200 #@param {type:"integer"}
tgt_seq_length = 200 #@param {type:"integer"}
idflag = False #@param {type:"boolean"}
t_ratio = 0.90 #@param {type:"number"}
save_checkpoint_steps = train_steps // 2

### train/test split

In [ ]:
train_test_split('train_mt_new.txt', 'train_pe_new.txt', 0.9, idflag)

# Configuration
The following script adds user-input to the required static configuration and outputs everything to yaml files.  
OpenNMT uses the yaml files to set its parameters:

- **preprocessing.yaml**: used in the preprocessing step.
- **training-config.yml**: used in the training step.

There is no configuration for the APE step.

In [ ]:
def configure():
    preprocessingyaml = [
    "train_src: mt_train.txt",
    "train_tgt: pe_train.txt",

    "valid_src: mt_test.txt",
    "valid_tgt: pe_test.txt",

    "save_data: prep-data",

    "src_vocab_size: 200000",
    "tgt_vocab_size: 200000",

    "shard_size: 100000",

    "bert_src: bert-base-multilingual-cased",
    "bert_tgt: bert-base-multilingual-cased",

    f"src_seq_length: {src_seq_length}",
    f"tgt_seq_length: {tgt_seq_length}"
    ]
    preprocessingyaml = "\n".join(preprocessingyaml)
    with open("preprocessing.yml", "w") as preprocessingyamlf:
        preprocessingyamlf.write(preprocessingyaml)
    trainyml = [
                "save_model: ape-model",

                "data: prep-data",
                f"train_steps: {train_steps}",
                f"start_decay_steps: {start_decay_steps}",
                f"valid_steps: {valid_steps}",
                f"save_checkpoint_steps: {save_checkpoint_steps}",
                f"keep_checkpoint: {keep_checkpoint}",

                "# Dimensionality",
                "rnn_size: 768 #!",
                "word_vec_size: 768 #!",
                "transformer_ff: 3072 #!",
                "heads: 12 #!",
                "layers: 12 #!",

                "# Embeddings",
                "position_encoding: 'true' #!",
                "share_embeddings: 'true' #!",
                "share_decoder_embeddings: 'true' #!",

                "# Encoder",
                "encoder_type: bert #!",
                "enc_bert_type: bert-base-multilingual-cased #!",

                "# Decoder",
                "decoder_type: bert #!",
                "dec_bert_type: bert-base-multilingual-cased #!",
                "bert_decoder_token_type: B #!",

                "# Layer Sharing",
                "bert_decoder_init_context: 'true'",
                "share_self_attn: 'true'",
                "# tie_context_attn: 'true'",
                "# share_feed_forward: 'true'",

                "# Regularization",
                "dropout: 0.1",
                "label_smoothing: 0.1",

                "# Optimization",
                "optim: bertadam #!",
                f"learning_rate: {learning_rate}",
                f"warmup_steps: {warmup_steps}",
                "batch_type: tokens",
                "normalization: tokens",
                "accum_count: 2",
                "batch_size: 512",
                "max_grad_norm: 0",
                "param_init: 0",
                "param_init_glorot: 'true'",
                "valid_batch_size: 8",

                f"average_decay: {average_decay}",

                "# GPU",
                "seed: 42",
                "world_size: 1",
                "gpu_ranks: 0",
    ]
    trainyml = "\n".join(trainyml)
    with open("train-config.yml", "w") as trainymlf:
        trainymlf.write(trainyml)

In [ ]:
configure()

# Run

### Preprocessing
This step converts the datasets to 
- **train**: prep-data-train.0.pt
- **valid**: prep-data-valid.0.pt
- **vocab**: prep-data-vocab.pt

In [ ]:
! python ./drive/MyDrive/OpenNMT-APE-master/preprocess.py -config preprocessing.yml

[2021-03-03 10:33:16,207 INFO] Extracting features...
[2021-03-03 10:33:16,207 INFO]  * number of source features: 0.
[2021-03-03 10:33:16,207 INFO]  * number of target features: 0.
[2021-03-03 10:33:16,207 INFO] Building `Fields` object...
[2021-03-03 10:33:16,207 WARNING] The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.
[2021-03-03 10:33:16,592 INFO] https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-vocab.txt not found in cache, downloading to /tmp/tmpxms88hb4
100% 995526/995526 [00:00<00:00, 1866317.68B/s]
[2021-03-03 10:33:17,530 INFO] copying /tmp/tmpxms88hb4 to cache at /root/.pytorch_pretrained_bert/96435fa287fbf7e469185f1062386e05a075cadbf6838b74da22bf64b080bc32.99bcd55fc66f4f3360bc49ba472b940b8dcf223ea6a345deb969d607ca900729
[2021-03-03 10:33:17,531 INFO] creating metadata file for /root/.pytorch_pretrained_be

### Training
The trainer takes in **{train, valid, vocab}** and produces a model.  
Models are named: **ape-model_step_X.pt**.  
**X** is the step number, the last is the best.

In [ ]:
try:
    ! python ./drive/MyDrive/OpenNMT-APE-master/train.py -config train-config.yml
except KeyboardInterrupt:
    sys.exit(0)

[2021-03-03 10:35:49,032 INFO]  * src vocab size = 119547
[2021-03-03 10:35:49,032 INFO]  * tgt vocab size = 119547
[2021-03-03 10:35:49,032 INFO] Building model...
[2021-03-03 10:35:58,453 INFO] https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased.tar.gz not found in cache, downloading to /tmp/tmp9hsklpkc
100% 662804195/662804195 [00:21<00:00, 30682394.84B/s]
[2021-03-03 10:36:20,600 INFO] copying /tmp/tmp9hsklpkc to cache at /root/.pytorch_pretrained_bert/731c19ddf94e294e00ec1ba9a930c69cc2a0fd489b25d3d691373fae4c0986bd.4e367b0d0155d801930846bb6ed98f8a7c23e0ded37888b29caa37009a40c7b9
[2021-03-03 10:36:21,532 INFO] creating metadata file for /root/.pytorch_pretrained_bert/731c19ddf94e294e00ec1ba9a930c69cc2a0fd489b25d3d691373fae4c0986bd.4e367b0d0155d801930846bb6ed98f8a7c23e0ded37888b29caa37009a40c7b9
[2021-03-03 10:36:21,532 INFO] removing temp file /tmp/tmp9hsklpkc
[2021-03-03 10:36:21,598 INFO] loading archive file https://s3.amazonaws.com/models.huggingfac

### Automatic Post Processing
- **input:** mt.txt
- **output:** ape.txt

The APE team did not provide documentation on how to use the model in production, and there is no script labeled as APE, however, their project is a fork of the original OpenNMT-py, which uses **translate.py** in production.  
Since APE is a translation of the form  **mt.txt -> ape.txt**, the same script can be used.  

**Note: The quality of APE will depend heavily on the training parameters, the longer the training the better.**  

 - A test of 10000 iterations took 6 hours of training.
 - A test of 50000 iterations will take around 30 Hours of training as an estimate.

In [ ]:
! echo using model: $(ls -v | grep -e 'ape-model_step' | tail -1)

using model: ape-model_step_50000.pt


In [ ]:
!python ./drive/MyDrive/OpenNMT-APE-master/translate.py -model $(ls -v | grep -e 'ape-model_step' | tail -1) -src ./multi_mt_60.txt -output ./drive/MyDrive/OpenNMT-APE-master/ape_multi_short_50k.txt -replace_unk -verbose -batch_size 8 -gpu 0

[2021-03-03 17:38:40,041 INFO] Translating shard 0.
/pytorch/aten/src/ATen/native/BinaryOps.cpp:66: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.

SENT 1: ['Z', '##nala', '##z', '##łem', '2', 'gry', 'print', '##i', 'i', 'ks', '##ię', '##ż', '##nic', '##zki']
PRED 1: Z ##nala ##z ##łem 2 print ##i i ks ##ię ##ż ##nic ##zki
PRED SCORE: -2.3830

SENT 2: ['20', 'minut', 'od', 'między', '##naro', '##dowego', 'port', '##u', 'lot', '##niczego']
PRED 2: 20 minut od Mi ##ędzy ##naro ##dowego Port ##u Lot ##niczego
PRED SCORE: -0.6983

SENT 3: ['W', 'wieku', '35', 'lat', 'oka', '##zja', 'jest', 'tylko', '1', 'na', '178', '##.']
PRED 3: W 35 latach oka ##zja jest tylko 1 na 178 ##.
PRED SCORE: -3.1979

SENT 4: ['0', '##.', '##1', 'G', '##ra', 'online', 'w', 'Ar', '##gent', '##ynie']
PRED 4: 0 ##. ##1 Ha ##zard online w Ar ##gent ##ynie
PRED SCORE:

In [ ]:
f = open('ape_multi.txt', 'r') 
lines = f.readlines()
f = open('ids.txt', 'r') 
idlines = f.readlines()
ids = idlines[0].split()
newlines = []
combine = False
i = 0
for line in lines:
  if combine:
    line = line.replace(' ##', '')
    newlines[-1] = newlines[-1][:len(newlines[-1])-2] + line
    combine = False
  else:
    if len(line) < 30:
      combine = True
    else:
      line = line.replace(' ##', '')
      newlines.append(ids[i] + ' ' + line)
      i += 1

outF = open('ape_multi.txt', 'w')
outF.writelines(newlines)
outF.close()

In [ ]:
from google.colab import files
files.download('ape_multi_10k.txt')

### Cleaning Up the APE file.

In [ ]:
cleanup("ape.txt", 'mt_test.txt')

{'missing lines': [], 'missing keys': []}


### Re-Process Missing Lines

In [ ]:
! python ./OpenNMT-APE/translate.py -model $(ls | grep -e 'ape-model_step_10000' | tail -1) -src missing_mt.txt -output missing_ape.txt -replace_unk -verbose -gpu 0

In [ ]:
cleanup("missing_ape.txt", 'missing_mt.txt')

{'missing lines': [], 'missing keys': []}
